# Resonance Mode Energy Transfer (RMET) Model

This notebook presents the **Resonance Mode Energy Transfer (RMET)** model, which proposes that mass and particle-like behavior emerge from resonance modes in a discretized fundamental field, as detailed in the accompanying LaTeX document.

We will explore:
1. The discretized 3D field setup
2. Stiffness operators and resonance modes
3. Minimalist formulation (massless nodes, \( M = I \))
4. Dispersion relations
5. Mode-aware source functionals and Green's operator
6. Emergent mass calculations
7. Field evolution visualization
8. Holonomy calculations for fundamental and composite modes
9. Numerical recipe and interpretation


## 1. Fundamental Field Setup

We define a discretized 3D cubic lattice with \( N_x \times N_y \times N_z \) nodes, representing the fundamental field at the Planck scale. Each node carries a scalar field degree of freedom \( u_i(t) \in \mathbb{R} \), and connectivity is represented by a stiffness matrix \( K = \alpha L \), where \( L \) is the 7-point discrete Laplacian.

The minimalist formulation sets the mass matrix to the identity: \( M = I \).


In [18]:
%pip install numpy

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh, expm
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import spsolve
import warnings
warnings.filterwarnings('ignore')

class ImprovedRMETModel:
    """
    Improved RMET implementation addressing key theoretical weaknesses:
    1. Physically motivated nonlinear source terms
    2. Realistic Berry connection via magnetic flux
    3. Better mass emergence mechanism
    4. Proper boundary conditions and convergence
    """
    
    def __init__(self, Nx, Ny, Nz, a=1.0, alpha=1.0, boundary='periodic'):
        self.Nx, self.Ny, self.Nz = Nx, Ny, Nz
        self.N = Nx * Ny * Nz
        self.a = a
        self.alpha = alpha
        self.boundary = boundary
        
        # Build stiffness matrix with proper boundary conditions
        self.K = self._build_stiffness_matrix()
        self.eigvals, self.eigvecs = eigh(self.K.toarray() if hasattr(self.K, 'toarray') else self.K)
        self.frequencies = np.sqrt(np.abs(self.eigvals))
        
        # Initialize modal amplitudes (complex for quantum-like behavior)
        self.amplitudes = np.zeros(self.N, dtype=complex)
        self.field = np.zeros(self.N, dtype=float)
        
        # Physical constants (in natural units where ℏ = c = 1)
        self.coupling_strength = 0.1  # Nonlinear coupling parameter
        self.damping = 0.01  # Lifetime/damping parameter
        
    def _build_stiffness_matrix(self):
        """Build 3D stiffness matrix with proper boundary conditions"""
        if self.boundary == 'periodic':
            return self._build_periodic_stiffness()
        elif self.boundary == 'absorbing':
            return self._build_absorbing_stiffness()
        else:
            return self._build_free_stiffness()
    
    def _build_periodic_stiffness(self):
        """Periodic boundary conditions - more suitable for bulk modes"""
        K = np.zeros((self.N, self.N))
        for i in range(self.Nx):
            for j in range(self.Ny):
                for k in range(self.Nz):
                    idx = self._get_index(i, j, k)
                    K[idx, idx] = 6 * self.alpha / self.a**2
                    
                    # Periodic neighbors
                    neighbors = [
                        ((i-1) % self.Nx, j, k),
                        ((i+1) % self.Nx, j, k),
                        (i, (j-1) % self.Ny, k),
                        (i, (j+1) % self.Ny, k),
                        (i, j, (k-1) % self.Nz),
                        (i, j, (k+1) % self.Nz)
                    ]
                    
                    for ni, nj, nk in neighbors:
                        neighbor_idx = self._get_index(ni, nj, nk)
                        K[idx, neighbor_idx] = -self.alpha / self.a**2
        return K
    
    def _build_absorbing_stiffness(self):
        """Absorbing boundary conditions - better for localized modes"""
        K = self._build_free_stiffness()
        # Add absorbing terms at boundaries (simplified PML)
        boundary_damping = 0.1 * self.alpha
        for i in range(self.N):
            coords = self._get_coordinates(i)
            if any(c == 0 or c == dim-1 for c, dim in zip(coords, [self.Nx, self.Ny, self.Nz])):
                K[i, i] += boundary_damping
        return K
    
    def _build_free_stiffness(self):
        """Free boundary conditions"""
        K = np.zeros((self.N, self.N))
        for i in range(self.Nx):
            for j in range(self.Ny):
                for k in range(self.Nz):
                    idx = self._get_index(i, j, k)
                    neighbors = []
                    
                    if i > 0: neighbors.append((i-1, j, k))
                    if i < self.Nx-1: neighbors.append((i+1, j, k))
                    if j > 0: neighbors.append((i, j-1, k))
                    if j < self.Ny-1: neighbors.append((i, j+1, k))
                    if k > 0: neighbors.append((i, j, k-1))
                    if k < self.Nz-1: neighbors.append((i, j, k+1))
                    
                    K[idx, idx] = len(neighbors) * self.alpha / self.a**2
                    for ni, nj, nk in neighbors:
                        neighbor_idx = self._get_index(ni, nj, nk)
                        K[idx, neighbor_idx] = -self.alpha / self.a**2
        return K
    
    def _get_index(self, i, j, k):
        """Convert 3D coordinates to 1D index"""
        return i + j * self.Nx + k * self.Nx * self.Ny
    
    def _get_coordinates(self, idx):
        """Convert 1D index to 3D coordinates"""
        k = idx // (self.Nx * self.Ny)
        j = (idx % (self.Nx * self.Ny)) // self.Nx
        i = idx % self.Nx
        return i, j, k

# Example: Compare boundary conditions
print("=== Boundary Condition Comparison ===")
models = {}
for boundary in ['periodic', 'free', 'absorbing']:
    models[boundary] = ImprovedRMETModel(4, 4, 4, boundary=boundary)
    print(f"{boundary:>10} - Lowest frequencies: {models[boundary].frequencies[:5]}")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
=== Boundary Condition Comparison ===
  periodic - Lowest frequencies: [5.96046448e-08 1.41421356e+00 1.41421356e+00 1.41421356e+00
 1.41421356e+00]
      free - Lowest frequencies: [3.16580421e-08 7.65366865e-01 7.65366865e-01 7.65366865e-01
 1.08239220e+00]
 absorbing - Lowest frequencies: [0.29514859 0.82585411 0.82585411 0.82585411 1.12715574]


## 2. Resonance Modes

The eigenmodes of the stiffness matrix \( K \) represent the natural resonance excitations of the lattice. The eigenvalue equation is:

\[ K \psi_n = \omega_n^2 \psi_n \]

We compute the eigenmodes and visualize the frequency spectrum.


In [19]:
# Enhanced mode analysis with boundary condition effects
def analyze_mode_structure(model, max_modes=10):
    """Analyze mode structure with physical interpretation"""
    
    print(f"Mode Analysis for {model.boundary} boundaries:")
    print("Mode |  Frequency  | Localization | Interpretation")
    print("-" * 55)
    
    for n in range(min(max_modes, len(model.frequencies))):
        freq = model.frequencies[n]
        mode = model.eigvecs[:, n]
        
        # Measure localization (inverse participation ratio)
        localization = np.sum(mode**4) / (np.sum(mode**2)**2)
        
        # Physical interpretation
        if freq < 1e-10:
            interpretation = "Zero mode (translation)"
        elif localization > 0.1:
            interpretation = "Localized state"
        elif freq / model.frequencies[-1] < 0.1:
            interpretation = "Low-energy bulk mode"
        else:
            interpretation = "High-energy mode"
        
        print(f"{n:4d} | {freq:10.6f} | {localization:11.6f} | {interpretation}")

# Analyze different boundary conditions
for boundary, model in models.items():
    analyze_mode_structure(model)
    print()

Mode Analysis for periodic boundaries:
Mode |  Frequency  | Localization | Interpretation
-------------------------------------------------------
   0 |   0.000000 |    0.015625 | Low-energy bulk mode
   1 |   1.414214 |    0.035546 | High-energy mode
   2 |   1.414214 |    0.031104 | High-energy mode
   3 |   1.414214 |    0.036578 | High-energy mode
   4 |   1.414214 |    0.037299 | High-energy mode
   5 |   1.414214 |    0.035936 | High-energy mode
   6 |   1.414214 |    0.041667 | High-energy mode
   7 |   2.000000 |    0.045814 | High-energy mode
   8 |   2.000000 |    0.047094 | High-energy mode
   9 |   2.000000 |    0.056449 | High-energy mode

Mode Analysis for free boundaries:
Mode |  Frequency  | Localization | Interpretation
-------------------------------------------------------
   0 |   0.000000 |    0.015625 | Low-energy bulk mode
   1 |   0.765367 |    0.025093 | High-energy mode
   2 |   0.765367 |    0.033735 | High-energy mode
   3 |   0.765367 |    0.033355 | High-e

## 2.a Physically Motivated Nonlinear Sources (replaces weak source implementation):

TODO: determine best integration with computer model and adjust numbering accordingly

In [20]:
def compute_physically_motivated_source(self, mode_indices=None, interaction_type='local'):
    """
    Compute physically motivated nonlinear source term.
    
    The source represents how modes couple nonlinearly:
    J[A] = Σ_n A_n ψ_n + g Σ_{p,q,r} A_p* A_q A_r ψ_p ψ_q ψ_r
    
    This captures:
    - Linear driving (first term)
    - Three-mode mixing (second term) - analogous to parametric processes
    """
    if mode_indices is None:
        mode_indices = range(min(10, self.N))  # Use first 10 modes
    
    # Linear term
    J_linear = np.dot(self.eigvecs[:, mode_indices], self.amplitudes[mode_indices])
    
    # Nonlinear term - three-mode coupling
    J_nonlinear = np.zeros(self.N, dtype=complex)
    
    if interaction_type == 'local':
        # Local cubic nonlinearity: |ψ|² ψ type
        for p in mode_indices:
            for q in mode_indices:
                for r in mode_indices:
                    if abs(self.frequencies[p] + self.frequencies[q] - self.frequencies[r]) < 0.1:  # Energy conservation
                        coupling = self.coupling_strength * np.sqrt(self.frequencies[p] * self.frequencies[q] / (self.frequencies[r] + 1e-10))
                        mode_product = self.eigvecs[:, p] * self.eigvecs[:, q] * self.eigvecs[:, r]
                        J_nonlinear += (coupling * np.conj(self.amplitudes[p]) * 
                                      self.amplitudes[q] * self.amplitudes[r] * mode_product)
    
    elif interaction_type == 'gradient':
        # Gradient-mediated coupling
        D = self._build_gradient_operator()
        for p in mode_indices:
            grad_p = D @ self.eigvecs[:, p]
            for q in mode_indices:
                grad_q = D @ self.eigvecs[:, q]
                coupling = self.coupling_strength * np.dot(grad_p, grad_q.conj()) / self.N
                J_nonlinear += coupling * self.amplitudes[p] * np.conj(self.amplitudes[q]) * self.eigvecs[:, q]
    
    return np.real(J_linear + J_nonlinear)

def _build_gradient_operator(self):
    """Build discrete gradient operator"""
    # Simplified 1D gradient along each direction
    D = np.zeros((3 * self.N, self.N))
    
    for i in range(self.Nx):
        for j in range(self.Ny):
            for k in range(self.Nz):
                idx = self._get_index(i, j, k)
                
                # X-gradient
                if i < self.Nx - 1:
                    D[idx, idx] = -1 / self.a
                    D[idx, self._get_index(i+1, j, k)] = 1 / self.a
                
                # Y-gradient  
                if j < self.Ny - 1:
                    D[self.N + idx, idx] = -1 / self.a
                    D[self.N + idx, self._get_index(i, j+1, k)] = 1 / self.a
                
                # Z-gradient
                if k < self.Nz - 1:
                    D[2 * self.N + idx, idx] = -1 / self.a
                    D[2 * self.N + idx, self._get_index(i, j, k+1)] = 1 / self.a
    
    return D

# Add these methods to ImprovedRMETModel class
ImprovedRMETModel.compute_physically_motivated_source = compute_physically_motivated_source
ImprovedRMETModel._build_gradient_operator = _build_gradient_operator

# Demonstrate nonlinear sources
model = ImprovedRMETModel(5, 5, 5)
model.amplitudes[:5] = [1.0, 0.5, 0.3, 0.1, 0.05]

print("=== Nonlinear Source Analysis ===")
J_local = model.compute_physically_motivated_source(list(range(5)), 'local')
J_gradient = model.compute_physically_motivated_source(list(range(5)), 'gradient')
print(f"Local coupling source RMS: {np.sqrt(np.mean(J_local**2)):.6f}")
print(f"Gradient coupling source RMS: {np.sqrt(np.mean(J_gradient**2)):.6f}")
print(f"Nonlinear/Linear ratio: {np.sqrt(np.mean(J_local**2)) / (np.sqrt(np.mean(model.amplitudes[:5]**2)) + 1e-10):.6f}")

=== Nonlinear Source Analysis ===
Local coupling source RMS: 0.104019
Gradient coupling source RMS: 0.104031
Nonlinear/Linear ratio: 0.200000+0.000000j


## 3. Dispersion Relation

The dispersion relation for the scalar-isotropic model is given by:

\[ \omega^2(\mathbf{k}) = \alpha \hat{L}(\mathbf{k}), \quad \hat{L}(\mathbf{k}) = \frac{2}{a^2} [3 - \cos(k_x a) - \cos(k_y a) - \cos(k_z a)] \]

For low \( k \), \( \omega(\mathbf{k}) \approx \sqrt{\alpha} |\mathbf{k}| \), with propagation speed \( c = \sqrt{\alpha} \).


In [21]:
def dispersion_relation(kx, ky, kz, a=1.0, alpha=1.0):
    """Compute dispersion relation for 3D cubic lattice."""
    L_hat = (2 / a**2) * (3 - np.cos(kx * a) - np.cos(ky * a) - np.cos(kz * a))
    return np.sqrt(alpha * L_hat)

# Generate k-points along a path (e.g., [0,0,0] to [pi/a,0,0])
k_max = np.pi / a
k_points = np.linspace(0, k_max, 100)
omega = dispersion_relation(k_points, 0, 0, a, alpha)

# Plot
plt.plot(k_points, omega, label=r'$\omega(k_x, 0, 0)$')
plt.xlabel(r'$k_x$')
plt.ylabel(r'$\omega$')
plt.title('Dispersion Relation for 3D Lattice')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'a' is not defined

## 4. Object-Oriented RMET Model

We treat resonance patterns as **objects** in an object-oriented programming (OOP) framework, packaging data (e.g., modal amplitudes, eigenmodes) and behaviors (e.g., propagation, interaction, holonomy).

- **Base Class (`ResonanceModeObject`)**: Defines core attributes (nodes, eigenmodes, amplitudes, frequencies) and methods (propagate, compute source, Green's operator, holonomy).
- **Subclasses**: `PhotonLikeMode` (massless dispersion) and `MassiveMode` (effective inertia from multi-node coupling).


In [ ]:
import numpy as np
from scipy.linalg import expm

class ResonanceModeObject:
    def __init__(self, nodes, K, amplitudes=None, lifetime=np.inf):
        self.nodes = nodes
        self.K = K
        # Compute eigenmodes and frequencies
        eigvals, eigvecs = np.linalg.eigh(K)
        self.frequencies = np.sqrt(np.abs(eigvals))
        self.eigenmodes = eigvecs
        self.amplitudes = np.array(amplitudes) if amplitudes is not None else np.zeros(len(eigvals), dtype=complex)
        self.lifetime = lifetime
        self.field = np.zeros(nodes, dtype=float)

    def set_initial_amplitudes(self, mode_indices, values):
        """Set initial modal amplitudes for specific modes."""
        for idx, val in zip(mode_indices, values):
            self.amplitudes[idx] = val
        return self

    def propagate(self, time_step=1.0):
        """Evolve modal amplitudes with phase e^(-i ω_n t) and compute field u(t)."""
        phases = np.exp(-1j * self.frequencies * time_step)
        self.amplitudes = self.amplitudes * phases
        self.field = np.real(np.dot(self.eigenmodes, self.amplitudes))
        return self

    def compute_source(self, Xi_pq=None, phi_pq=None):
        """Compute mode-aware source J[A](x,t).
        Xi_pq is a (N_modes, N_modes) matrix.
        phi_pq is a (N_modes, N_modes, N_nodes) tensor.
        """
        J = np.dot(self.eigenmodes, self.amplitudes)  # Linear term
        if Xi_pq is not None and phi_pq is not None:
            # Quadratic term: sum_pq A_p^* A_q Xi_pq phi_pq[p, q]
            quad_term = np.zeros(self.nodes, dtype=complex)
            N_modes = len(self.amplitudes)
            for p in range(N_modes):
                for q in range(N_modes):
                    quad_term += np.conj(self.amplitudes[p]) * self.amplitudes[q] * Xi_pq[p, q] * phi_pq[p, q]
            J += quad_term
        return np.real(J)

    def greens_operator(self, omega, Sigma=None):
        """Compute retarded Green's operator G(ω)."""
        if Sigma is None:
            Sigma = np.zeros_like(self.K)
        return np.linalg.inv(self.K - omega**2 * np.eye(self.nodes) + Sigma)

    def medium_response(self, omega, J):
        """Compute medium displacement W(ω) = G(ω) J."""
        G = self.greens_operator(omega)
        return np.dot(G, J)

    def discrete_gradient_operator(self, Nx, Ny, Nz, a=1.0):
        """Build discrete gradient operator D for 3D lattice."""
        N = Nx * Ny * Nz
        D = np.zeros((3 * N, N))
        for i in range(Nx):
            for j in range(Ny):
                for k in range(Nz):
                    idx = i + j * Nx + k * Nx * Ny
                    if i < Nx - 1:
                        D[idx, idx] = -1 / a
                        D[idx, idx + 1] = 1 / a
                    if j < Ny - 1:
                        D[N + idx, idx] = -1 / a
                        D[N + idx, idx + Nx] = 1 / a
                    if k < Nz - 1:
                        D[2 * N + idx, idx] = -1 / a
                        D[2 * N + idx, idx + Nx * Ny] = 1 / a
        return D

    def effective_mass_gradient(self, Nx, Ny, Nz, a=1.0):
        """Compute m_eff using gradient-based method."""
        D = self.discrete_gradient_operator(Nx, Ny, Nz, a)
        Psi_comp = np.dot(self.eigenmodes, self.amplitudes)
        return np.dot(Psi_comp, np.dot(D.T @ D, Psi_comp))

    def compute_berry_connection(self, theta, mode_indices, Nx, Ny, Nz, a=1.0, alpha=1.0):
        """Compute Berry connection A_ij = <ψ_i | ∇_θ | ψ_j> for selected modes using fixed numerical derivative."""
        delta_theta = 1e-4
        N_modes = len(mode_indices)
        A = np.zeros((N_modes, N_modes), dtype=complex)
        # Rotate K by theta (simplified: rotate coupling terms in x-direction)
        K_theta = build_3d_stiffness(Nx, Ny, Nz, a, alpha)
        # Perturb K by rotating coupling terms (example: rotate x-direction couplings)
        K_theta_plus = build_3d_stiffness(Nx, Ny, Nz, a, alpha * np.cos(theta + delta_theta))
        eigvals, eigvecs = np.linalg.eigh(K_theta)
        eigvals_plus, eigvecs_plus = np.linalg.eigh(K_theta_plus)
        for i_idx, i in enumerate(mode_indices):
            for j_idx, j in enumerate(mode_indices):
                psi_i = eigvecs[:, i]
                psi_j = eigvecs[:, j]
                psi_j_plus = eigvecs_plus[:, j]
                grad_psi_j = (psi_j_plus - psi_j) / delta_theta
                A[i_idx, j_idx] = np.dot(psi_i.conj(), grad_psi_j)
        return A

    def compute_holonomy(self, mode_indices, Nx, Ny, Nz, a=1.0, alpha=1.0):
        """Compute holonomy D(2π) by path-ordered integration of Berry connection over θ = 0 to 2π."""
        theta_points = np.linspace(0, 2 * np.pi, 100)
        delta_theta = theta_points[1] - theta_points[0]
        D = np.eye(len(mode_indices), dtype=complex)
        for theta in theta_points[:-1]:  # Integrate up to last point
            A = self.compute_berry_connection(theta, mode_indices, Nx, Ny, Nz, a, alpha)
            # Use matrix exponential for each infinitesimal step
            D_step = expm(1j * A * delta_theta)
            D = np.dot(D_step, D)  # Left-multiply for path-ordering
        return D

    def plot_field_evolution(self, timesteps):
        """Plot evolution of field u(t) over time."""
        fields = []
        for t in timesteps:
            self.propagate(time_step=t)
            fields.append(self.field)
        
        plt.figure(figsize=(10, 6))
        for i, field in enumerate(fields):
            plt.plot(field, label=f"t={timesteps[i]:.2f}")
        plt.xlabel("Node Index")
        plt.ylabel(r"Field $u(t)$")
        plt.title("Field Evolution Over Time")
        plt.legend()
        plt.grid(True)
        plt.show()

class PhotonLikeMode(ResonanceModeObject):
    def __init__(self, frequency, amplitude=1.0, nodes=1):
        K = np.array([[frequency**2]])  # Single-mode stiffness
        super().__init__(nodes=nodes, K=K, amplitudes=[amplitude])
    
    def propagate(self, time_step=1.0, c=1.0):
        """Phase evolution for massless dispersion."""
        self.amplitudes *= np.exp(-1j * self.frequencies * time_step / c)
        self.field = np.real(np.dot(self.eigenmodes, self.amplitudes))
        return self

class MassiveMode(ResonanceModeObject):
    def __init__(self, mass, frequency, amplitude=1.0, nodes=10):
        K = np.array([[frequency**2]])  # Simplified for single mode
        super().__init__(nodes=nodes, K=K, amplitudes=[amplitude])
        self.mass = mass
    
    def propagate(self, time_step=1.0):
        """Phase evolution slowed by effective mass."""
        self.amplitudes *= np.exp(-1j * self.frequencies * time_step / (1 + self.mass))
        self.field = np.real(np.dot(self.eigenmodes, self.amplitudes))
        return self

# Example usage
K = build_3d_stiffness(Nx=3, Ny=3, Nz=3)
mode = ResonanceModeObject(nodes=Nx*Ny*Nz, K=K)
mode.set_initial_amplitudes(mode_indices=[0, 1], values=[1.0+0j, 0.5+0j])
mode.propagate(time_step=0.1)
print("Field u(t):", mode.field)


## 5. Mode-Aware Source Functional

The mode-aware source is defined as:

\[ J[\mathbf{A}](x,t) = \sum_n A_n(t) \psi_n(x) + \sum_{p,q} (\mathbf{A}^\dagger \Xi^{(pq)}(x) \mathbf{A}) \varphi_{pq}(x) \]

Here, \( \Xi_{pq} \) is a (N_modes, N_modes) matrix representing weights, and \( \varphi_{pq} \) is a (N_modes, N_modes, N_nodes) tensor for local mode products, e.g., \( \varphi_{pq}(x) = \psi_p(x) \psi_q(x) \).

We compute this source for a simple diagonal \( \Xi^{(pq)} \) and local \( \varphi_{pq} \).


In [ ]:
# Compute source J[A]
N_modes = len(mode.amplitudes)
Xi_pq = np.eye(N_modes)  # (N_modes, N_modes) weight matrix
phi_pq = np.zeros((N_modes, N_modes, mode.nodes))
for p in range(N_modes):
    for q in range(N_modes):
        phi_pq[p, q] = mode.eigenmodes[:, p] * mode.eigenmodes[:, q]
J = mode.compute_source(Xi_pq=Xi_pq, phi_pq=phi_pq)
print("Source J[A]:", J)


## 6. Green's Operator and Medium Response

The medium displacement \( W(\omega) \) is computed as:

\[ (K - \omega^2 I + \Sigma(\omega)) W(\omega) = J[\mathbf{A}](\omega) \]

The retarded Green's operator is:

\[ G(\omega) = (K - \omega^2 I + \Sigma(\omega))^{-1} \]

TODO: determine if the math changes due to the code changes below


In [ ]:
def improved_greens_function(self, omega, mode_indices=None, include_self_energy=True):
    """
    Improved Green's function with physical self-energy corrections.
    
    G(ω) = (K - ω²I + Σ(ω))⁻¹
    
    Self-energy includes:
    - Damping from mode coupling
    - Boundary effects
    - Nonlinear corrections
    """
    if mode_indices is None:
        K_reduced = self.K
        N_modes = self.N
    else:
        K_reduced = self.K[np.ix_(mode_indices, mode_indices)]
        N_modes = len(mode_indices)
    
    # Base Green's function
    omega_matrix = omega**2 * np.eye(N_modes)
    
    if include_self_energy:
        # Self-energy corrections
        Sigma = np.zeros((N_modes, N_modes), dtype=complex)
        
        # Damping term
        Sigma += 1j * self.damping * omega * np.eye(N_modes)
        
        # Mode coupling corrections (simplified)
        if mode_indices is not None:
            for i, mi in enumerate(mode_indices):
                for j, mj in enumerate(mode_indices):
                    if i != j:
                        coupling = self.coupling_strength * np.abs(self.amplitudes[mi] * self.amplitudes[mj])
                        Sigma[i, j] += coupling * omega / (self.frequencies[mi] + self.frequencies[mj] + 1e-10)
        
        G_inv = K_reduced - omega_matrix + Sigma
    else:
        G_inv = K_reduced - omega_matrix
    
    # Regularize near singularities
    try:
        G = np.linalg.inv(G_inv)
    except np.linalg.LinAlgError:
        # Add small regularization
        reg = 1e-10 * np.eye(N_modes)
        G = np.linalg.inv(G_inv + reg)
    
    return G

# Add method to class
ImprovedRMETModel.improved_greens_function = improved_greens_function

# Demonstrate Green's function improvements
print("=== Improved Green's Function Analysis ===")
omega_test = model.frequencies[2]
G_simple = model.improved_greens_function(omega_test, include_self_energy=False)
G_improved = model.improved_greens_function(omega_test, include_self_energy=True)

print(f"Test frequency: {omega_test:.6f}")
print(f"Simple G condition number: {np.linalg.cond(G_simple):.2e}")
print(f"Improved G condition number: {np.linalg.cond(G_improved):.2e}")
print(f"Improvement ratio: {np.linalg.cond(G_simple)/np.linalg.cond(G_improved):.2f}")

# Plot Green's function vs frequency
frequencies_test = np.linspace(0.1, 2.0, 100)
G_diagonal_simple = []
G_diagonal_improved = []

for omega in frequencies_test:
    try:
        G_s = model.improved_greens_function(omega, [0, 1, 2], False)
        G_i = model.improved_greens_function(omega, [0, 1, 2], True)
        G_diagonal_simple.append(np.abs(G_s[0, 0]))
        G_diagonal_improved.append(np.abs(G_i[0, 0]))
    except:
        G_diagonal_simple.append(np.inf)
        G_diagonal_improved.append(np.inf)

plt.figure(figsize=(10, 6))
plt.semilogy(frequencies_test, G_diagonal_simple, 'b-', label='Simple G(ω)')
plt.semilogy(frequencies_test, G_diagonal_improved, 'r-', label='Improved G(ω)')
plt.xlabel('Frequency ω')
plt.ylabel('|G₀₀(ω)|')
plt.title("Green's Function: Simple vs Improved")
plt.legend()
plt.grid(True)
plt.ylim(1e-3, 1e3)
plt.show()

## 7. Emergent Mass

### Rigorous Derivation of Mass Emergence Mechanism

In the RMET model, mass emerges from the spatial structure of resonance modes in the discretized field. For a composite mode \( \Psi_{\text{comp}} = \sum_p A_p \psi_p \), the effective mass \( m_{\text{eff}} \) quantifies the 'inertia' due to spatial gradients.

Starting from the kinetic energy term in the continuum limit, the effective mass is related to the integral of the gradient squared:

\[ T = \frac{1}{2} \int \dot{u}^2 dV \approx \frac{1}{2} m_{\text{eff}} \dot{v}^2 \]

where \( v \) is the 'velocity' of the mode center. For discretized fields, this becomes:

\[ m_{\text{eff}} = \Psi_{\text{comp}}^\top D^\top D \Psi_{\text{comp}} \]

Here, \( D \) is the discrete gradient operator, capturing how mode amplitude variations across nodes contribute to effective inertia. In the minimalist formulation (\( M = I \)), node masses are zero, and \( m_{\text{eff}} > 0 \) arises purely from mode delocalization and coupling.

For photon-like modes (low k, delocalized), \( m_{\text{eff}} \approx 0 \). For massive modes (localized or composite), higher gradients lead to \( m_{\text{eff}} > 0 \), slowing propagation as seen in the modified dispersion \( \omega \approx \sqrt{k^2 + m_{\text{eff}}^2} \).


In [ ]:
def compute_emergent_mass_improved(self, mode_indices, mass_type='kinetic'):
    """
    Improved mass calculation with multiple physical interpretations.
    
    Three approaches:
    1. Kinetic: m = ∫ |∇ψ|² (classical kinetic energy analog)
    2. Binding: m = binding energy from mode overlaps
    3. Effective: m from modified dispersion relation
    """
    if not mode_indices:
        return 0.0
    
    # Construct composite wavefunction
    psi_composite = np.zeros(self.N, dtype=complex)
    for idx in mode_indices:
        psi_composite += self.amplitudes[idx] * self.eigvecs[:, idx]
    
    if mass_type == 'kinetic':
        # Kinetic mass: m = ∫ |∇ψ|²
        D = self._build_gradient_operator()
        grad_psi = D @ psi_composite
        return np.real(np.sum(np.abs(grad_psi)**2))
    
    elif mass_type == 'binding':
        # Binding mass from mode coupling matrix
        binding_matrix = np.zeros((len(mode_indices), len(mode_indices)))
        for i, p in enumerate(mode_indices):
            for j, q in enumerate(mode_indices):
                # Overlap integral weighted by frequency difference
                overlap = np.sum(self.eigvecs[:, p] * self.eigvecs[:, q])
                freq_factor = abs(self.frequencies[p] - self.frequencies[q])
                binding_matrix[i, j] = overlap * freq_factor
        
        eigenvals = np.linalg.eigvals(binding_matrix)
        return np.max(np.real(eigenvals))  # Largest binding energy
    
    elif mass_type == 'effective':
        # Effective mass from inverse curvature of dispersion
        if len(mode_indices) == 1:
            n = mode_indices[0]
            # Approximate second derivative of ω(k) at mode n
            k_eff = np.sqrt(self.eigvals[n] / self.alpha)
            if k_eff > 0:
                return self.alpha / (k_eff**2 + 1e-10)  # Inverse curvature
        return 0.0
    
    return 0.0

# Add method to class
ImprovedRMETModel.compute_emergent_mass_improved = compute_emergent_mass_improved

# Comprehensive mass analysis
print("=== Enhanced Mass Emergence Analysis ===")
mode_sets = [
    ([0], "Ground state"),
    ([1], "First excited"),
    ([0, 1], "Ground doublet"),
    ([1, 2, 3], "Excited triplet"),
    ([0, 1, 2, 3, 4], "Low-energy quintuplet")
]

for modes, description in mode_sets:
    print(f"\n{description} {modes}:")
    mass_kinetic = model.compute_emergent_mass_improved(modes, 'kinetic')
    mass_binding = model.compute_emergent_mass_improved(modes, 'binding')
    mass_effective = model.compute_emergent_mass_improved(modes, 'effective') if len(modes) == 1 else 0
    
    print(f"  Kinetic mass:   {mass_kinetic:.6f}")
    print(f"  Binding mass:   {mass_binding:.6f}")
    if len(modes) == 1:
        print(f"  Effective mass: {mass_effective:.6f}")

# Visualize mass vs mode number
mode_numbers = range(1, min(20, model.N))
masses_kinetic = [model.compute_emergent_mass_improved([n], 'kinetic') for n in mode_numbers]
masses_effective = [model.compute_emergent_mass_improved([n], 'effective') for n in mode_numbers]

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.plot(mode_numbers, masses_kinetic, 'bo-', label='Kinetic mass')
plt.xlabel('Mode number')
plt.ylabel('Mass')
plt.title('Kinetic Mass vs Mode Number')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(mode_numbers, masses_effective, 'ro-', label='Effective mass')
plt.xlabel('Mode number')
plt.ylabel('Mass')
plt.title('Effective Mass vs Mode Number')
plt.grid(True)
plt.tight_layout()
plt.show()

## 8. Holonomy Calculations

Holonomy calculations assess projective mode-space rotations for degenerate doublets, aiming to verify \( D(2\pi) = -I \). The Berry connection is computed as:

\[ A_{ij}(\theta) = \langle \psi_i(\theta) | \nabla_\theta | \psi_j(\theta) \rangle \]

The holonomy is:

\[ D(2\pi) = \mathcal{P} \exp \left( i \oint A(\theta) \, d\theta \right) \]

We compute this for a doublet or composite mode subspace.

TODO: determine if the descriptive changes due to the 'physically grounded approach'


In [ ]:
def compute_berry_connection_magnetic(self, mode_indices, flux_quantum=0.1):
    """
    Compute Berry connection via magnetic flux threading.
    
    More realistic than arbitrary stiffness rotation:
    - Thread magnetic flux through lattice
    - Compute how eigenmodes change
    - Extract Berry connection from phase changes
    """
    n_modes = len(mode_indices)
    n_flux_points = 50
    flux_values = np.linspace(0, 2*np.pi, n_flux_points)
    
    berry_connection = np.zeros((n_flux_points-1, n_modes, n_modes), dtype=complex)
    
    for i, flux in enumerate(flux_values[:-1]):
        # Compute K with magnetic flux
        K_flux = self._add_magnetic_flux(flux, flux_quantum)
        eigvals_flux, eigvecs_flux = eigh(K_flux)
        
        # Next flux point
        K_flux_next = self._add_magnetic_flux(flux_values[i+1], flux_quantum)
        eigvals_next, eigvecs_next = eigh(K_flux_next)
        
        # Compute Berry connection A_ij = <ψ_i | d/dφ | ψ_j>
        d_flux = flux_values[i+1] - flux
        for ii, mi in enumerate(mode_indices):
            for jj, mj in enumerate(mode_indices):
                psi_i = eigvecs_flux[:, mi]
                psi_j = eigvecs_flux[:, mj]
                psi_j_next = eigvecs_next[:, mj]
                
                # Ensure consistent phase convention
                overlap = np.dot(psi_j.conj(), psi_j_next)
                if np.abs(overlap) > 1e-10:
                    psi_j_next *= np.sign(np.real(overlap))
                
                dpsi_j = (psi_j_next - psi_j) / d_flux
                berry_connection[i, ii, jj] = np.dot(psi_i.conj(), dpsi_j)
    
    return berry_connection

def _add_magnetic_flux(self, flux, flux_quantum):
    """Add magnetic flux through Peierls substitution"""
    K_flux = self.K.copy()
    
    # Add flux-dependent phases to x-direction bonds
    # Simplified: flux creates phase factors on horizontal bonds
    for j in range(self.Ny):
        for k in range(self.Nz):
            phase = np.exp(1j * flux * flux_quantum * j / self.Ny)
            for i in range(self.Nx - 1):
                idx1 = self._get_index(i, j, k)
                idx2 = self._get_index(i+1, j, k)
                if isinstance(K_flux, np.ndarray):
                    # Apply phase to hopping terms
                    original_coupling = -self.alpha / self.a**2
                    K_flux[idx1, idx2] = original_coupling * np.real(phase)
                    K_flux[idx2, idx1] = original_coupling * np.real(phase)
    
    return K_flux

def compute_holonomy_improved(self, mode_indices, flux_quantum=0.1):
    """
    Compute holonomy via path-ordered exponential of Berry connection.
    
    More numerically stable implementation with proper path ordering.
    """
    berry_A = self.compute_berry_connection_magnetic(mode_indices, flux_quantum)
    n_steps = len(berry_A)
    
    # Path-ordered exponential
    holonomy = np.eye(len(mode_indices), dtype=complex)
    
    for i in range(n_steps):
        d_flux = 2 * np.pi / n_steps
        # Matrix exponential of infinitesimal Berry connection
        A_step = 1j * berry_A[i] * d_flux
        U_step = expm(A_step)
        holonomy = U_step @ holonomy
    
    return holonomy

# Add methods to class
ImprovedRMETModel.compute_berry_connection_magnetic = compute_berry_connection_magnetic
ImprovedRMETModel._add_magnetic_flux = _add_magnetic_flux
ImprovedRMETModel.compute_holonomy_improved = compute_holonomy_improved

# Demonstrate improved holonomy calculation
print("=== Realistic Berry Connection and Holonomy ===")
try:
    # Test with different mode pairs
    mode_pairs = [(0, 1), (1, 2), (2, 3)]
    
    for pair in mode_pairs:
        print(f"\nModes {pair}:")
        holonomy = model.compute_holonomy_improved(list(pair))
        det_holonomy = np.linalg.det(holonomy)
        
        print(f"  Holonomy matrix:")
        print(f"    {holonomy[0,0]:.4f}  {holonomy[0,1]:.4f}")
        print(f"    {holonomy[1,0]:.4f}  {holonomy[1,1]:.4f}")
        print(f"  Determinant: {det_holonomy:.6f}")
        print(f"  |det(H)|: {np.abs(det_holonomy):.6f}")
        print(f"  Phase of det: {np.angle(det_holonomy):.6f}")
        print(f"  Close to -1? {np.abs(np.abs(det_holonomy) - 1) < 0.1}")
        
        # Check if it's close to a projective rotation
        if np.abs(det_holonomy + 1) < 0.1:
            print("  → Potential spinor-like behavior!")
        elif np.abs(det_holonomy - 1) < 0.1:
            print("  → Scalar-like behavior")
        else:
            print("  → Complex projective behavior")

except Exception as e:
    print(f"Holonomy calculation error: {e}")
    print("This may indicate numerical instability or inappropriate mode selection")


## 9. Field Evolution Visualization

We visualize the time evolution of the field \( \mathbf{u}(t) = \sum_{n=1}^{N_m} \Re\{A_n(t) \psi_n e^{-i \omega_n t}\} \).


In [ ]:
# Visualize field evolution
timesteps = np.linspace(0, 1.0, 5)
mode.plot_field_evolution(timesteps)


## 10. Numerical Recipe for Minimalist RMET

Following the LaTeX document's recipe:
1. **Assemble K**: Implemented in `build_3d_stiffness`.
2. **Solve eigenproblem**: Included in `ResonanceModeObject.__init__`.
3. **Construct J[A]**: Implemented in `compute_source`.
4. **Compute G(ω)**: Implemented in `greens_operator`.
5. **Solve (W, A) system**: Partially implemented in `medium_response` and `propagate`.
6. **Construct composites and compute m_eff**: Implemented in `effective_mass_gradient`.
7. **Measure holonomy**: Implemented in `compute_holonomy`.
8. **Validate convergence**: To be added (e.g., test with larger lattices or PML boundaries).


## 11. Convergence Tests and Validation

To validate the model, we perform convergence tests by increasing lattice size and compare numerical frequencies to analytical dispersion in the low-k limit.

For a larger lattice (e.g., 5x5x5), check if effective mass stabilizes and dispersion matches \( \omega \approx \sqrt{\alpha} k \).


In [ ]:
# Convergence test: Larger lattice
Nx_l, Ny_l, Nz_l = 5, 5, 5
K_large = build_3d_stiffness(Nx_l, Ny_l, Nz_l, a, alpha)
mode_large = ResonanceModeObject(nodes=Nx_l*Ny_l*Nz_l, K=K_large)
mode_large.set_initial_amplitudes(mode_indices=[0, 1], values=[1.0+0j, 0.5+0j])
m_eff_large = mode_large.effective_mass_gradient(Nx_l, Ny_l, Nz_l, a)
print("Effective Mass (5x5x5):", m_eff_large)

# Validation against analytical dispersion
# For lowest mode, k ~ pi / (N a), omega ~ sqrt(alpha) * k
k_approx = np.pi / (Nx * a)
omega_analytical = np.sqrt(alpha) * k_approx
omega_numerical = mode.frequencies[1]  # First non-zero mode
print("Analytical omega (low k):", omega_analytical)
print("Numerical omega (mode 1):", omega_numerical)
print("Relative error:", abs(omega_analytical - omega_numerical) / omega_analytical)


## 12. Interpretation of Minimalist RMET

- **Uniform Inertia**: Setting \( M = I \) simplifies dynamics, with emergent mass arising from mode structure and spatial gradients.
- **Physical Behavior**: The model captures dispersion, polarization, composite binding, and projective rotations without explicit node mass.
- **Holonomy**: Non-trivial holonomy (\( D(2\pi) = -I \)) indicates spinor-like behavior, suggesting emergent quantum-like properties.
- **Limitations**: Quantum statistics require additional mechanisms (e.g., topological or quantization approaches).


## 13. Exporting to LaTeX

To generate a LaTeX document from this notebook:
```bash
jupyter nbconvert --to latex RMET_Model_Notebook_OOPV2.ipynb
```
Compile the resulting `.tex` file with `pdflatex`. Ensure equations use `\begin{equation}...\end{equation}` for proper numbering during export.


## 14. Summary and Next Steps

- Defined a 3D discretized field and stiffness operator.
- Derived resonance modes, dispersion relations, and holonomy.
- Implemented minimalist formulation with \( M = I \).
- Computed mode-aware sources, Green's operator, emergent mass, and holonomy.
- Visualized field evolution.
- Added convergence tests and mass derivation.

**Next steps**:
- Implement nonlinear couplings and higher-order resonances.
- Refine holonomy calculations with realistic K(θ) parameterizations.
- Explore mapping to Standard Model structures.
- Validate with larger lattices and boundary conditions.
